# Criação do dataset para o Teste 0: variando a SNR e a quantidade de usuários ao mesmo tempo

## Imports

In [1]:
import sys
sys.path.append("../../python/")

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import pandas as pd
import utils
from typing import Optional
from pathlib import Path
from joblib import Parallel, delayed
from time import time
from IPython.display import display, clear_output

## Funções

In [2]:
def simularPropagacaoSinal(
    qtdSimbolosPiloto: int, 
    bitsPorSimbolo:    int, 
    qtdAntenas:        int, 
    qtdUsuarios:       int, 
    potenciaEspiao:    float, 
    snr:               float, 
    qtdEspioes:        int
) -> (np.ndarray, np.ndarray, np.ndarray):
    
    # SIMULANDO OS CANAIS DO USUARIO E DO ESPIAO
    Haut = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)) + 1j*np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)))
    g    = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdEspioes))  + 1j*np.random.normal(0, 1, size=(qtdAntenas, qtdEspioes)))

    # SEQUENCIA PILOTO ALEATORIA + MODULACAO PARA TODOS OS USUARIOS:
    bitStream = np.random.choice([0, 1], qtdSimbolosPiloto*bitsPorSimbolo*qtdUsuarios)
    symb      = utils.qpskmodulator(bitStream) # QPSK Modulator
    xp        = symb.reshape(qtdUsuarios, qtdSimbolosPiloto)

    # ESPIAO ENTRANDO NA JOGADA (SE A POTENCIA DELE FOR 0 ELE NAO ENTRA NA JOGADA):
    xpe  = np.sqrt(potenciaEspiao)*xp[0, :] # xpe vai ser a sequencia piloto do primeiro usuario multiplicada pela raiz da potencia do espiao
    xptx = np.concatenate((xp, [xpe])) # xptx sera a matriz xp com uma linha a mais: xpe
    H    = np.concatenate((Haut, g), axis=1) # H vai ser Haut com uma COLUNA a mais, que vai ser g

    # TRANSMISSAO PELO CANAL
    Y = np.dot(H, xptx) # fading
    Y = utils.awgn(Y, SNR=snr) # ruido branco
    
    return Y, xp, H[:,0]

In [3]:
def estimarCanal(
    Y:  np.ndarray, 
    xp: np.ndarray
) -> np.ndarray:
    Hest = np.matmul(
        np.matmul(Y, np.conjugate(xp).T), 
        np.linalg.lstsq(
            np.matmul(xp, np.conjugate(xp).T), 
            np.eye(
                np.matmul(xp, np.conjugate(xp).T).shape[0], 
                np.matmul(xp, np.conjugate(xp).T).shape[0]
            )
        )[0]
    )
    return Hest[:,0]

In [4]:
def calcularEnergiaHassan(
    HestCol0:          np.ndarray, 
    snr:               float, 
    qtdAntenas:        int, 
    qtdSimbolosPiloto: int
) -> (float, float):
    
    N0       = 1/(10**(snr/10))
    sovertau = qtdAntenas*N0/qtdSimbolosPiloto
    ln       = np.log((2+sovertau)/(1+sovertau))
    eta      = ((1 + sovertau)*(2+sovertau)*ln).real
    E        = (np.matmul(np.conjugate(HestCol0).T, HestCol0)/qtdAntenas).real
    
    return E, eta

In [5]:
def calcularFeatures(
    qtdSimbolos:    int, 
    bitsPorSimbolo: int, 
    qtdAntenas:     int, 
    qtdUsuarios:    int, 
    potenciaEspiao: float, 
    snr:            float, 
    qtdEspioes:     int
) -> (float, float):
    
    # PROPAGACAO DO SINAL
    Y, xp, HCol0 = simularPropagacaoSinal(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes)  
    
    # ESTIMATIVA DO CANAL (hassan)
    HestCol0 = estimarCanal(Y, xp)
    
    # CALCULO DA ENERGIA (hassan)
    E, eta = calcularEnergiaHassan(HestCol0, snr, qtdAntenas, qtdSimbolos)
    
    return E, eta

In [6]:
def gerarNovaAmostra(
    qtdSimbolos:    int, 
    bitsPorSimbolo: int, 
    qtdAntenas:     int, 
    qtdUsuarios:    int, 
    potenciaEspiao: int, 
    snr:            float, 
    qtdEspioes:     int, 
    caminhoCSV:     Path
) -> None:
    
    # OBTENDO AS PRINCIPAIS FEATURES
    E, eta = calcularFeatures(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes)

    # FAZENDO O TARGET DO DATASET
    ataquePresente = True if potenciaEspiao else False

    # SALVANDO NO CSV
    linhaNovaCSV = pd.DataFrame([[
        qtdUsuarios,
        snr,
        E,
        eta,
        potenciaEspiao, 
        ataquePresente
    ]]).to_csv(caminhoCSV, mode="a", header=False, index=False)

In [7]:
def gerarDataset(
    rangePotenciaEspiao: np.ndarray,
    rangeQtdUsuarios:    np.ndarray,
    rangeSNRs:           np.ndarray,
    qtdAntenas:          int, 
    qtdSimbolos:         int, 
    qtdBitsPorSimbolo:   int, 
    qtdEspioes:          int, 
    repetibilidade:      int,
    caminhoCSV:          Path
) -> None:
    
    # INICIANDO O CSV
    colunas = ["qtdUsuarios", "SNR", "E", "eta", "potenciaEspiao", "ataquePresente"]
    dataframe = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)

    # PARA PRINTAR O PROGRESSO
    iteracaoAtual     = 1 
    qtdPossibilidades = len(rangeQtdUsuarios) * len(rangeSNRs) * len(rangePotenciaEspiao)

    # ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS
    for qtdUsuarios in rangeQtdUsuarios:
        for snr in rangeSNRs:
            for potenciaEspiaoAtual in rangePotenciaEspiao:

                # PARALELIZANDO
                Parallel(n_jobs=nJobs, verbose=0)(delayed(gerarNovaAmostra)(qtdSimbolos, qtdBitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiaoAtual, snr, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))            

                # PRINT
                printStr  = "Qtd usuários:       " + str(qtdUsuarios) + "\n"
                printStr += "SNR:                " + str(snr) + "\n"
                printStr += "Potência do Espião: " + str(potenciaEspiaoAtual) + "\n"
                printStr += "Progresso Total:    " + str(100*(iteracaoAtual/qtdPossibilidades))[:7] + "%"
                clear_output(wait=True)
                print(printStr)
                iteracaoAtual += 1

## Parâmetros Iniciais

In [8]:
# PARAMETROS FIXOS
nJobs                 = 6
qtdAntenas            = 256
qtdSimbolos           = 300
qtdEspioes            = 1
qtdBitsPorSimbolo     = 2
repetibilidade        = 100
potenciaUsuario       = 1
sufixoNomeCSV         = str(time()).replace(".", "")
dirDatasets           = Path("../../datasets/test_0/")
caminhoCSVTreinamento = dirDatasets.joinpath("train_0_"+str(qtdAntenas)+"_antennas_"+sufixoNomeCSV+".csv")
caminhoCSVTeste       = dirDatasets.joinpath("test_0_"+str(qtdAntenas)+"_antennas_"+sufixoNomeCSV+".csv")

# PARAMETROS VARIAVEIS
rangePotenciaEspiao         = np.arange(0, 2.51, 0.5)
rangeQtdUsuariosTreinamento = np.concatenate(([1], np.arange(16, 257, 16)))
rangeQtdUsuariosTeste       = rangeQtdUsuariosTreinamento
rangeSNRsTreinamento        = np.arange(-10, 31, 5)
rangeSNRsTeste              = rangeSNRsTreinamento

## Balanceamento do Dataset

Queremos que a quantidade de cenários em que há ataque seja igual à quantidade de casos em que não há. Se o array de potências do espião for [0.0, 0.5, 1.0, 1.5, 2.0, 2.5], teremos 5x mais dados em que há contaminação. Nesse caso, temos que fazer o array se transformar em [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5]. 

In [9]:
rangePotenciaEspiao = np.concatenate((np.zeros(np.count_nonzero(rangePotenciaEspiao>0)-np.count_nonzero(rangePotenciaEspiao==0)), rangePotenciaEspiao))
print(rangePotenciaEspiao)

[0.  0.  0.  0.  0.  0.5 1.  1.5 2.  2.5]


## Criando os dataset

### Treinamento

In [10]:
gerarDataset(
    rangePotenciaEspiao,
    rangeQtdUsuariosTreinamento,
    rangeSNRsTreinamento,
    qtdAntenas,
    qtdSimbolos,
    qtdBitsPorSimbolo,
    qtdEspioes,
    repetibilidade,
    caminhoCSVTreinamento
) 

Qtd usuários:       256
SNR:                30
Potência do Espião: 2.5
Progresso Total:    100.0%


### Teste

In [ ]:
gerarDataset(
    rangePotenciaEspiao,
    rangeQtdUsuariosTeste,
    rangeSNRsTeste,
    qtdAntenas,
    qtdSimbolos,
    qtdBitsPorSimbolo,
    qtdEspioes,
    repetibilidade, 
    caminhoCSVTeste
) 

Qtd usuários:       176
SNR:                15
Potência do Espião: 2.5
Progresso Total:    68.6274%
